In [24]:
import tensorflow as tf
import numpy as np 
import extract_dataset as dat
import time_series_const as time
import post_text_preprocess as pro
import tfidf as tfidf
import json
import pickle

###################################################################################
#                                                                                 #
#       DATASET EXTRACTION                                                        #
#                                                                                 #
###################################################################################
# 4664 labeled events 
# For the labels, the value is 1 if the event is a rumor, and is 0 otherwise. 
# The content of all the posts in are in json format (timestamp and text)
# where each file is named event_id.json, corresponding to individual event

n_ev = 4664 # number of evenements
#event_ids = np.zeros((n_ev,1),dtype=int) #events ids
#labels = np.zeros((n_ev,1),dtype=int) # labels 0 or 1
event_ids = np.zeros((n_ev,1)) #events ids
labels = np.zeros((n_ev,1),dtype=int) # labels 0 or 1

# Extract labels and corresponding event ids
event_related_posts = dat.extract_dataset(event_ids,labels,"Weibo.txt",n_ev)


#Split the data into training and testing sets 80/20
N_test = int(0.2 * event_related_posts.shape[0])

#event_related_posts_train = event_related_posts[N_test:event_related_posts.shape[0],:]
labels_train= labels[N_test:labels.shape[0],:]
event_ids_train = event_ids[N_test:event_ids.shape[0],:]

#event_related_posts_test = event_related_posts[0:N_test,:]
labels_test= labels[0:N_test,:]
event_ids_test = event_ids[0:N_test,:]


    
rnn_data_train=[] #time series tfidf RRN input data for each event
rnn_data_test=[]

## load event time series txt files (to save time)
with open("training_event_time_series_tfidf_N30.txt", "rb") as fp:   # Unpickling
          rnn_data_train=(pickle.load(fp))        
with open("test_event_time_series_tfidf_N30.txt", "rb") as fp:   # Unpickling
          rnn_data_test=(pickle.load(fp))
          
          
# Keep only the K-most important score per interval
# pad the number of intervals (each event need same number of intervals)
# put each event in a numpy array

  





    



In [25]:
#Reshuffling the data to get a more even distribution of rumor and non-rumor event across training and test data

import random
reshuffled_labels = labels
reshuffled_data = rnn_data_train.copy()
reshuffled_data.extend(rnn_data_test)
temp = list(zip(reshuffled_labels, reshuffled_data))

random.shuffle(temp)
reshuffled_labels, reshuffled_data = zip(*temp)
reshuffled_labels = np.array(reshuffled_labels)

labels_train= reshuffled_labels[N_test:reshuffled_labels.shape[0],:]
labels_test= reshuffled_labels[0:N_test,:]

rnn_data_train = [sublist for sublist in reshuffled_data[N_test:]]
rnn_data_test = [sublist for sublist in reshuffled_data[0:N_test]]

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(labels_train)
labels_train = enc.transform(labels_train).toarray()
enc.fit(labels_test)
labels_test = enc.transform(labels_test).toarray()


print(labels_train.shape)
print(labels_test.shape)

(3732, 2)
(932, 2)


In [26]:
#Just checking what the max number of tf.idf values (maxK) inside any interval in the data is
maxK = 0
for event in rnn_data_train:
    maxK = max(len(max(event,key=len)),maxK)
print("Largest K value in all intervals = " + str(maxK))
maxNrIntervals = max( len(max(rnn_data_train,key=len)), len(max(rnn_data_test,key=len)))
print("Largest # intervals in a single event = " + str(maxNrIntervals))

Largest K value in all intervals = 389739
Largest # intervals in a single event = 58


In [27]:
# k = 100 #the number of tf.idf values sorted in descending order we will keep for each interval
# maxNrIntervals = 30 #equivalent to N value 

# print(maxNrIntervals)
# new_rnn_train = []
# new_rnn_test = []
# #Processing Training Data
# for event in rnn_data_train:
#     new_event = []
#     for interval in event: 
#         kInterval = sorted(interval, reverse=True)[:k]
#         kInterval.extend([0]*(k-len(kInterval))) #append the interval with zeros until it has a length of k
#         new_event.append(kInterval)
#         if len(new_event) == maxNrIntervals: break
#     while len(new_event) < maxNrIntervals:
#         new_event.append([0]*k) #append the event with intervals of zeros until it has a length of maxNrIntervals
#     new_rnn_train.append(new_event)

# #Processing Test Data
# for event in rnn_data_test:
#     new_event = []
#     for interval in event: 
#         kInterval = sorted(interval, reverse=True)[:k]
#         kInterval.extend([0]*(k-len(kInterval))) #append the interval with zeros until it has a length of k
#         new_event.append(kInterval)
#         if len(new_event) == maxNrIntervals: break
#     while len(new_event) < maxNrIntervals:
#         new_event.append([0]*k) #append the event with intervals of zeros until it has a length of maxNrIntervals
#     new_rnn_test.append(new_event)
        
# new_rnn_train = np.array(new_rnn_train) #convert the standard python lists to numpy arrays
# norm = np.linalg.norm(new_rnn_train)
# new_rnn_train = new_rnn_train/norm
# new_rnn_test = np.array(new_rnn_test)
# new_rnn_test = new_rnn_test/norm
# print(new_rnn_train.shape)
# print(new_rnn_test.shape)

In [34]:
import sys

k = 5000 #the number of tf.idf values sorted in descending order we will keep for each interval
maxNrIntervals = 30 #equivalent to N value 

print(maxNrIntervals)
new_rnn_train = []
new_rnn_test = []
#Processing Training Data
print("Start for train")
for idx, event in enumerate(rnn_data_train):
    sys.stdout.write("Progress: %d%%   \r" % (idx/len(rnn_data_train)*100) )
    sys.stdout.flush()
    new_event = []
    for interval in event: 
        highestVals = (-np.array(interval)).argsort()[:k]
        kInterval = []
        for i, val in enumerate(interval):
            if (i in highestVals):
               kInterval.append(val) 
        kInterval.extend([0]*(k-len(kInterval))) #append the interval with zeros until it has a length of k 
        new_event.append(kInterval)
        if len(new_event) == maxNrIntervals: break
    while len(new_event) < maxNrIntervals:
        new_event.append([0]*k) #append the event with intervals of zeros until it has a length of maxNrIntervals
    new_rnn_train.append(new_event)

#Processing Test Data
print("Start for test")
for idx, event in enumerate(rnn_data_test):
    sys.stdout.write("Progress: %d%%   \r" % (idx/len(rnn_data_test)*100) )
    sys.stdout.flush()
    new_event = []
    for interval in event: 
        highestVals = (-np.array(interval)).argsort()[:k]
        kInterval = []
        for i, val in enumerate(interval):
            if (i in highestVals):
               kInterval.append(val) 
        kInterval.extend([0]*(k-len(kInterval))) #append the interval with zeros until it has a length of k
        new_event.append(kInterval)
        if len(new_event) == maxNrIntervals: break
    while len(new_event) < maxNrIntervals:
        new_event.append([0]*k) #append the event with intervals of zeros until it has a length of maxNrIntervals
    new_rnn_test.append(new_event)
        
new_rnn_train = np.array(new_rnn_train) #convert the standard python lists to numpy arrays
norm = np.linalg.norm(new_rnn_train)
new_rnn_train = new_rnn_train/norm
new_rnn_test = np.array(new_rnn_test)
new_rnn_test = new_rnn_test/norm
print(new_rnn_train.shape)
print(new_rnn_test.shape)

30
Start for train
Start for test  
(3732, 30, 5000)
(932, 30, 5000)


In [32]:
new_rnn_train[0]

array([[3.17646808e-03, 5.83095667e-04, 5.39647913e-04, ...,
        1.94365222e-04, 3.02915842e-04, 3.02915842e-04],
       [1.55889018e-04, 1.38132261e-04, 1.38132261e-04, ...,
        1.81870521e-04, 3.22308608e-04, 3.22308608e-04],
       [1.65028287e-03, 1.65028287e-03, 4.00019122e-04, ...,
        2.75047144e-04, 2.75047144e-04, 2.00009561e-04],
       ...,
       [3.75187919e-05, 3.75187919e-05, 3.75187919e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [30]:
tf.device('/gpu:1') #My best gpu is gpu:1, change to gpu:0 if you only have 1 gpu
CUDA_VISIBLE_DEVICES=1 

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, SimpleRNN

model = Sequential()

model.add(SimpleRNN(maxNrIntervals, activation='tanh',use_bias=True, kernel_initializer='uniform',
                   recurrent_initializer='orthogonal', kernel_regularizer=tf.keras.regularizers.l2(l=1),
                    bias_initializer='zeros',dropout=0.0, recurrent_dropout=0.0,
                   return_sequences=False))
model.add(Dense(2,activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(l=1)))

opt = tf.keras.optimizers.Adagrad(lr=0.1, initial_accumulator_value=0.9, epsilon=1e-07)
#opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)
model.compile(
    loss='categorical_crossentropy', 
    #this should be mse between the probability distributions of the prediction and ground truth + L2-regularization penalty
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(new_rnn_train,
          labels_train,
          epochs=10,
          batch_size=32,
          validation_data=(new_rnn_test, labels_test))

Train on 3732 samples, validate on 932 samples
Epoch 1/10
3732/3732 [==============================] - 4s 1ms/sample - loss: 0.8747 - accuracy: 0.5027 - val_loss: 0.6951 - val_accuracy: 0.4936
Epoch 2/10
3732/3732 [==============================] - 2s 606us/sample - loss: 0.6936 - accuracy: 0.5056 - val_loss: 0.6946 - val_accuracy: 0.5064
Epoch 3/10
3732/3732 [==============================] - 2s 598us/sample - loss: 0.6939 - accuracy: 0.5083 - val_loss: 0.6931 - val_accuracy: 0.5064
Epoch 4/10
3732/3732 [==============================] - 2s 608us/sample - loss: 0.6937 - accuracy: 0.4992 - val_loss: 0.6934 - val_accuracy: 0.5064
Epoch 5/10
3732/3732 [==============================] - 2s 605us/sample - loss: 0.6938 - accuracy: 0.4922 - val_loss: 0.6933 - val_accuracy: 0.5064
Epoch 6/10
3732/3732 [==============================] - 2s 604us/sample - loss: 0.6936 - accuracy: 0.5024 - val_loss: 0.6935 - val_accuracy: 0.5064
Epoch 7/10
3732/3732 [==============================] - 2s 610us/sa